In [1]:
import csv
import numpy as np
from numpy import random
import sys
from SwingyMonkey import SwingyMonkey
import itertools
import matplotlib.pyplot as plt
from collections import defaultdict
#%matplotlib inline

https://www.bcs.rochester.edu/people/robbie/jacobslab/cheat_sheet/ModelBasedRL.pdf

http://mlg.eng.cam.ac.uk/mlss09/mlss_slides/Littman_1.pdf

http://outlace.com/Reinforcement-Learning-Part-3/

## Model Based

##### Description

The main idea behind the model based system is to estimate and maximize : Rtotal(s, a)/N(s, a)
Which is the average reward of an action (a) taken in state (s). In our specific case the state can be defined by the various parameter obtain from the game (tree, monkey):
    
    tree{dist, top, bot)
    monkey{vel, top, bot}
    
One has to implement a sort of "grid" in order to go through all the potential state, taking each potential pixel would be tedious, it then make sense to define hyperparameters to define the grid size.

    x_grid_px
    y_grid_px

##### Main Parameters

In [2]:
x_grid_px = 20
y_grid_px = 20

##### Class

In [3]:
class ModelBasedLearner:

    def __init__(self, gamma):
        self.previous_state  = None
        self.previous_action = None
        self.previous_reward = None
        self.screen_height = 400
        self.screen_width = 600
        
        self.gamma = gamma
        
        # Get total number of states given the grid 
        self.total_states = ((self.screen_width/x_grid_px)*2 + 1) * ((self.screen_height/y_grid_px)*2 + 1)
        
        # Keep track of all states while running
        self.list_of_states = [] #To store list of all States
        
        ### New Initialisation (Following Mark)
        self.Q                   = defaultdict(lambda: [0,0]) #Q[States][Action]
        self.R                   = defaultdict(lambda: [[0,1] , [0,1]]) # R[States][action][n,tot]
        self.P                   = defaultdict(lambda: [[0,1] , [0,1]]) # R[States_prev, states][action][n,tot]
        

    def convert_state(self, state):
        binsize = 50
        dist_thresh    = 50
        speed_thresh   = 0
        
        monkey_tree_top_dist = (state['monkey']['top'] - state['tree']['top']) / binsize   
        #monkey_tree_bot_dist = (state['monkey']['bot'] - state['tree']['bot'])/ binsize

        monkey_near_top      = (self.screen_height - state['monkey']['top']) < dist_thresh
        monkey_near_bottom   = state['monkey']['bot']                 < dist_thresh
        monkey_near_tree     = state['tree']['dist']                  < dist_thresh
        monkey_fast          = state['monkey']['vel']                 > speed_thresh
        return (monkey_near_top, monkey_near_bottom, monkey_near_tree, monkey_fast, monkey_tree_top_dist)


    def reset(self):
        self.previous_state  = None
        self.previous_action = None
        self.previous_reward = None

    # computes optimal policy and value function value at a given state
    def get_best_value(self, state):   
        if self.Q[state][1] > self.Q[state][0]:
            return (self.Q[state][1],1)
        else:
            return (self.Q[state][0],0)
    
    def UpdateT(self, state_i):
        self.P[self.previous_state, state_i][self.previous_action][0] += 1
        # Update Count
        for state in self.list_of_states:
            self.P[self.previous_state, state][self.previous_action][1] += 1
                
    def UpdateR(self):
        self.R[self.previous_state][self.previous_action][0] += self.previous_reward
        # Update Count
        self.R[self.previous_state][self.previous_action][1] += 1
        
    def GetProbability(self, previous_state, state, action):
        # Prob = N / ToT
        return self.P[previous_state,state][action][0] / self.P[previous_state,state][action][1]
    
    def GetAverageR(self, state, action):
        return self.R[state][action][0] / self.R[state][action][1]
        
    def UpdateQ(self, state_i):
        
        # Update Q function (action = 0)
        prob_x_Expectation = 0.0
        for state in self.list_of_states:
            prob_x_Expectation += (self.GetProbability(state_i, state, 0) * self.get_best_value(state)[0])
        self.Q[state_i][0] = self.GetAverageR(state_i, 0) + (self.gamma * prob_x_Expectation)

        # Update Q function (action = 1)
        prob_x_Expectation = 0.0
        for state in self.list_of_states:
            prob_x_Expectation += (self.GetProbability(state_i,state,1) * self.get_best_value(state)[0])
        self.Q[state_i][1] = self.GetAverageR(state_i, 1) + (self.gamma * prob_x_Expectation)
             
    
    def action_callback(self, state):
        
        #state_i = State(state['tree']['dist'],state['monkey']['bot'], state['tree']['bot'])
        state_i = self.convert_state(state)

        if self.previous_state != None:
            self.UpdateT(state_i)
            self.UpdateR()
            self.UpdateQ(state_i)

            # Decide which action
            if self.Q[state_i][0] < self.Q[state_i][1]:
                self.previous_action = 1
            else:
                self.previous_action = 0
                
        else:
            self.previous_action = random.random() < 0.1

        #Memory Save Previous state
        self.previous_state = state_i

        return self.previous_action

    def reward_callback(self, reward):
        self.previous_reward = reward

In [4]:
class State:
    def __init__(self, hdistance_to_trunk, monkey_height, lower_trunk_height):
        self.x = np.floor(hdistance_to_trunk*1./x_grid_px)
        self.y = np.floor( (monkey_height -lower_trunk_height)*1. /y_grid_px)

    def __hash__(self):
        return int( (self.x+30)*100+(self.y+20) )

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y 

    def __repr__(self):
        return "(%d,%d)" % (self.x, self.y)

In [8]:
def run_games(learner, hist, iters = 100, t_len = 1):
    for ii in range(iters):
        # Make a new monkey object.
        swing = SwingyMonkey(sound=False,                  # Don't play sounds.
                             text="Epoch %d" % (ii),       # Display the epoch on screen.
                             tick_length = t_len,          # Make game ticks super fast.
                             action_callback=learner.action_callback,
                             reward_callback=learner.reward_callback)

        # Loop until you hit something.
        while swing.game_loop():
            pass
        
        # Save score history.
        hist.append(swing.score)

        # Reset the state of the learner.
        learner.reset()
    return

In [9]:
def run(gamma):
    agent = ModelBasedLearner(gamma)
    scores = []
    run_games(agent, scores) 
    #np.save('scores' + '_' + str(alpha) + '_' + str(gamma), np.array(scores))
    return scores

## Results Analysis

In [10]:
all_scores = dict()
gammas = [0.25, 0.5, 0.75, 1]
for gamma in gammas:
    scores = run(gamma)
    all_scores[gamma] = scores

In [17]:
import pandas as pd
score_df = pd.DataFrame(columns=['gamma','total','max_score'])
for i,(k,v) in enumerate(all_scores.iteritems()):
    gamma = k
    total =  sum(v)
    max_score = max(v)
    score_df.loc[i] = (gamma, total, max_score)

In [18]:
score_df

,gamma,total,max_score
0,0.25,144.0,12.0
1,0.50,151.0,16.0
2,0.75,171.0,14.0
3,1.00,139.0,14.0
